<a href="https://colab.research.google.com/github/thevirusoup/thevirusoup/blob/main/DATA_HORSE_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyplink


In [ ]:
import pyplink


In [ ]:
from __future__ import print_function, division

In [ ]:
from pyplink import PyPlink

In [ ]:
pedfile = PyPlink(r'/home/hp/Documents/ML/Horse_AIMS')

In [ ]:
print("{:,d} samples and {:,d} markers".format(
    pedfile.get_nb_samples(),
    pedfile.get_nb_markers(),
))

In [ ]:
all_samples = pedfile.get_fam()
all_samples.head()

In [ ]:
all_markers = pedfile.get_bim()
all_markers.head()

In [ ]:
# Additive format: Cycling through genotypes as -1, 0, 1 and 2 values, where -1 is unknown, 0 is homozygous (major allele), 1 is heterozygous, and 2 is homozygous (minor allele).
import numpy as np

# Initialize empty lists to store marker_id and genotypes
marker_ids_list = []
genotypes_list = []

# Iterate over marker_id and genotypes in pedfile
for marker_id, genotypes in pedfile:
    # Append marker_id and genotypes to the respective lists
    marker_ids_list.append(marker_id)
    genotypes_list.append(genotypes)

# Convert the lists into numpy arrays
marker_ids_array = np.array(marker_ids_list)
genotypes_array = np.array(genotypes_list)

# Transpose the genotypes_array to have genotypes in rows
genotypes_array = np.transpose(genotypes_array)

matrix = np.vstack((marker_ids_array, genotypes_array))

# Save the matrix to a text file
np.savetxt('Horse_try.txt', matrix, delimiter='\t', fmt='%s')


In [ ]:
import pandas as pd

# Load the main data file
main_file = 'Horse_try.txt'
main_df = pd.read_csv(main_file, delimiter='\t')

# Load the new column data file, including its header
new_column_file = 'labels_horse.txt'
new_column_df = pd.read_csv(new_column_file, delimiter='\t')

# Extract the column name and values from the new column DataFrame
new_column_name = new_column_df.columns[0]
new_column_values = new_column_df[new_column_name]

# Insert the new column at the first position (index 0)
main_df.insert(0, new_column_name, new_column_values)

# Save the modified DataFrame to a .csv file
output_file = 'Horse_try.csv'
main_df.to_csv(output_file, index=False)

print(f'New column from "{new_column_file}" has been added as the first column and saved to {output_file}.')


In [ ]:
#Nucleotide format Cycling through genotypes as A, C, G and T values (where 00 is unknown)
import numpy as np

# Initialize empty lists to store marker_id and genotypes
marker_ids_list = []
genotypes_list = []

# Iterate over marker_id and genotypes in pedfile
for marker_id, genotypes in pedfile.iter_acgt_geno():
      # Append marker_id and genotypes to the respective lists
    marker_ids_list.append(marker_id)
    genotypes_list.append(genotypes)

# Convert the lists into numpy arrays
marker_ids_array = np.array(marker_ids_list)
genotypes_array = np.array(genotypes_list)

# Transpose the genotypes_array to have genotypes in rows
genotypes_array = np.transpose(genotypes_array)

matrix = np.vstack((marker_ids_array, genotypes_array))

# Save the matrix to a text file
np.savetxt('RP_N_matrix.txt', matrix, delimiter='\t', fmt='%s')


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer

# Read the CSV file into a pandas DataFrame
file_path = r"C:\Users\kkoka\Desktop\Python learning\RP_DFI_80%AC_encoded.csv"
data = pd.read_csv(file_path)

# Separate the features (SNPs) and the target variable (breed)
features = data.iloc[:, 1:]  # Exclude the first column (breed)
target = data.iloc[:, 0]  # First column (breed)

# Impute missing values with the mean
imputer = SimpleImputer(strategy='mean')
imputed_features = imputer.fit_transform(features)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(imputed_features, target, test_size=0.25, random_state=42)

# Create and train the Random Forest Classifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

# Read the CSV file into a pandas DataFrame
file_path = r"C:\Users\kkoka\Desktop\Python learning\RP_DFI_80%AC_encoded.csv"
data = pd.read_csv(file_path)

# Separate the features (SNPs) and the target variable (breed)
features = data.iloc[:, 1:]  # Exclude the first column (breed)
target = data.iloc[:, 0]  # First column (breed)

# Create a Random Forest Classifier
clf = RandomForestClassifier()

# Define the number of folds for cross-validation
n_splits = 5

# Perform stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

accuracies = []  # To store accuracy for each fold

# Iterate over the folds
for train_index, test_index in skf.split(features, target):
    # Split the data into training and testing sets for the current fold
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]

    # Perform Recursive Feature Elimination with Cross-Validation (RFECV)
    rfecv = RFECV(estimator=clf, step=1, cv=skf)
    rfecv.fit(X_train, y_train)

    # Get the selected feature indices and names
    selected_indices = rfecv.get_support(indices=True)
    selected_features = X_train.columns[selected_indices]

    # Train the classifier using the selected features
    clf.fit(X_train[selected_features], y_train)

    # Make predictions on the test set using the selected features
    y_pred = clf.predict(X_test[selected_features])

    # Calculate the accuracy for the current fold
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracies) / n_splits

# Get the selected features with the highest accuracy
best_indices = rfecv.get_support(indices=True)
best_features = features.columns[best_indices]

# Print the selected features
print("Selected Features:")
print(best_features)

# Check if accuracy exceeds 90%
if average_accuracy > 0.9:
    # Save the selected features to a text file
    with open("selected_features.txt", "w") as file:
        for feature in best_features:
            file.write(feature + "\n")
    print("Selected features saved to selected_features.txt")
else:
    print("Unable to achieve accuracy > 90% with the selected features.")


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score

# Read the CSV file into a pandas DataFrame
file_path = r"C:\Users\kkoka\Desktop\Python learning\RP_DFI_80%AC_encoded.csv"
data = pd.read_csv(file_path)

# Separate the features (SNPs) and the target variable (breed)
features = data.iloc[:, 1:]  # Exclude the first column (breed)
target = data.iloc[:, 0]  # First column (breed)

# Perform missing value imputation
imputer = SimpleImputer(strategy='mean')
imputed_features = pd.DataFrame(imputer.fit_transform(features), columns=features.columns)

# Create a Random Forest Classifier
clf = RandomForestClassifier()

# Define the number of splits for cross-validation
n_splits = 3

# Perform stratified shuffle-split cross-validation
sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=0.2, random_state=42)

accuracies = []  # To store accuracy for each split

# Iterate over the splits
for train_index, test_index in sss.split(imputed_features, target):
    # Split the data into training and testing sets for the current split
    X_train, X_test = imputed_features.iloc[train_index], imputed_features.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]

    # Perform feature selection using SelectKBest with chi2 scoring
    selector = SelectKBest(score_func=f_classif, k=70)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    # Train the classifier using the selected features
    clf.fit(X_train_selected, y_train)

    # Make predictions on the test set using the selected features
    y_pred = clf.predict(X_test_selected)

    # Calculate the accuracy for the current split
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)

# Calculate the average accuracy across all splits
average_accuracy = sum(accuracies) / n_splits

# Get the selected feature indices
selected_indices = selector.get_support(indices=True)

# Get the selected features with the highest accuracy
best_features = imputed_features.columns[selected_indices]

# Print the selected features
print("Selected Features:")
print(best_features)

# Check if accuracy exceeds 85%
if average_accuracy > 0.85:
    # Save the selected features to a text file
    with open("selected_features2.txt", "w") as file:
        for feature in best_features:
            file.write(feature + "\n")
    print("Selected features saved to selected_features2.txt")
else:
    print("Unable to achieve accuracy > 85% with the selected features.")


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score

# Read the CSV file into a pandas DataFrame
file_path = r"C:\Users\kkoka\Desktop\Python learning\RP_DFI_80%AC_encoded.csv"
data = pd.read_csv(file_path)

# Separate the features (SNPs) and the target variable (breed)
features = data.iloc[:, 1:]  # Exclude the first column (breed)
target = data.iloc[:, 0]  # First column (breed)

# Perform missing value imputation
imputer = SimpleImputer(strategy='mean')
imputed_features = pd.DataFrame(imputer.fit_transform(features), columns=features.columns)

# Perform feature selection using SelectKBest with chi2 scoring
selector = SelectKBest(score_func=f_classif, k=100)
selected_features = selector.fit_transform(imputed_features, target)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(selected_features, target, test_size=0.3, random_state=42)

# Create a Random Forest Classifier
clf = RandomForestClassifier()

# Train the classifier using the selected features
clf.fit(X_train, y_train)

# Make predictions on the test set using the selected features
y_pred = clf.predict(X_test)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Get the selected feature indices
selected_indices = selector.get_support(indices=True)

# Get the selected features
selected_snps = imputed_features.columns[selected_indices]

# Save the selected SNPs to a text file
with open("selected_snps.txt", "w") as file:
    for snp in selected_snps:
        file.write(snp + "\n")

print("Selected SNPs saved to selected_snps.txt")
